题目描述：分析题目附件中的代码，根据服务端所给出的counter和token，伪造出一个可以通过验证的counter和token，并提交至服务端  
题目附件中的代码：

In [ ]:
import binascii
from gmssl import sm3, func
from Crypto.Util.number import *

In [ ]:
## 服务端代码

# 读取HMAC key文件
def read_hmac_key(file_path):
    with open(file_path, 'rb') as f:
        hmac_key = f.read().strip()
    return hmac_key
    
# 生成token
def generate_token(hmac_key, counter):
    # 如果HMAC_KEY长度不足32字节，则在末尾补0，超过64字节则截断
    if len(hmac_key) < 32:
        hmac_key = hmac_key.ljust(32, b'\x00')
    elif len(hmac_key) > 32:
        hmac_key = hmac_key[:32]
    # 将计数器转换为字节表示
    counter_bytes = counter.to_bytes((counter.bit_length() + 7) // 8, 'big')
    # print("counter_bytes:", binascii.hexlify(counter_bytes))
    tobe_hashed = bytearray(hmac_key + counter_bytes)
    # print("tobe_hashed:", binascii.hexlify(tobe_hashed))
    # 使用SM3算法计算哈希值
    sm3_hash = sm3.sm3_hash(tobe_hashed)
    # 将SM3的哈希值转换为十六进制字符串作为token
    token = sm3_hash
    return token

current_counter = 0
def verify_token(hmac_key, counter, token):
    # 生成token
    generated_token = generate_token(hmac_key, counter)
    global current_counter
    # 比较生成的token和输入的token是否相同
    if generated_token == token:
        if counter & 0xFFFFFFFF > current_counter:
            current_counter = counter & 0xFFFFFFFF
            print("current_counter: ", hex(current_counter))
            return "Success"
        else:
            return "Error: counter must be increasing"
    else:
        return "Error: token not match"

# 假设HMAC key文件路径
hmac_key_file = 'hmac_key.txt'
# 读取HMAC key
hmac_key = read_hmac_key(hmac_key_file)

In [ ]:
# 假设计数器值
counter = 0x12345678

# 生成token
token = generate_token(hmac_key, counter)
print("Generated token:", token)
print(verify_token(hmac_key, counter, token))

现在需要根据上面得出的token和counter，伪造出新的token和counter，并通过验证   
答案与解析如下：

In [ ]:
token = int(token, 16)
print("Token in int:", token)

counter_new = bytes.fromhex(hex(counter)[2:]) 
counter_new += b'\x80' + b'\x00' * 19    # padding
counter_new += b'\x00' * 6 + b'\x01\x20' # len
counter_new += b'\xff' * 4               # counter_append
last_block = b'\xff' * 4
last_block += b'\x80' + b'\x00' * 51     # padding
last_block += b'\x00' * 6 + b'\x02\x20'  # len

prefHashValue = bytes.fromhex(hex(token)[2:])
prefHashValue = [bytes_to_long(prefHashValue[i:i+4]) for i in range(0, 32, 4)]
NewHashValue = sm3.sm3_cf(prefHashValue, func.bytes_to_list(last_block))
NewHashValue = ''.join(['%08x'%val for val in NewHashValue])

print('counter:', counter_new.hex())
print('token:', NewHashValue)

In [ ]:
## 通过服务端的接口进行验证
# 注意，进行过一次长度扩展攻击后，再次提交验证，
# 会出现Error: counter must be increasing
# 因为全局变量 current_counter (确保counter递增)
# 再次运行该单元格可查看结果
print(verify_token(hmac_key, int.from_bytes(counter_new, byteorder='big'), NewHashValue))
## 自验证
generated_token = generate_token(hmac_key, int.from_bytes(counter_new, 'big'))
print(generated_token == NewHashValue)

相关资料：   
[长度拓展攻击](https://www.cnblogs.com/yunen/p/13624595.html#:~:text=%E5%AF%B9%E4%BA%8EHASH%E9%95%BF%E5%BA%A6%E6%8B%93%E5%B1%95%E6%94%BB%E5%87%BB%E7%9A%84%E7%AE%80%E5%8D%95%E5%AD%A6%E4%B9%A0%E7%AC%94%E8%AE%B0)   